In [16]:
"""
Fine‑tuning YOLOv8 en COCO128, 100 % reproducible
-------------------------------------------------
- Descarga COCO128
- Prepara estructura YOLO
- Crea YAML de configuración
- Entrena (fine‑tuning) congelando capas
- Valida e infiere
"""

import os, zipfile, requests, yaml, shutil, random
from pathlib import Path
from tqdm import tqdm
import torch
from ultralytics import YOLO
import yaml

# ¿Qué capas hay?

In [9]:
model = YOLO("yolo11n.pt")          # el que uses
print(model.model)                  # lista legible de módulos


DetectionModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (2): C3k2(
      (cv1): Conv(
        (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(48, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
   

In [12]:
layers = list(model.model.model.children())
print(layers)

[Conv(
  (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
  (act): SiLU(inplace=True)
), Conv(
  (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
  (act): SiLU(inplace=True)
), C3k2(
  (cv1): Conv(
    (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
    (act): SiLU(inplace=True)
  )
  (cv2): Conv(
    (conv): Conv2d(48, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
    (act): SiLU(inplace=True)
  )
  (m): ModuleList(
    (0): Bottleneck(
      (cv1): Conv(
        (conv): Conv2d(16, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)

In [10]:
layers = list(model.model.model.children())  # accede a la secuencia de capas

for i, layer in enumerate(layers):
    n_params = sum(p.numel() for p in layer.parameters())
    print(f"{i:>2}: {layer.__class__.__name__:<10} | {n_params:,} parámetros")

 0: Conv       | 464 parámetros
 1: Conv       | 4,672 parámetros
 2: C3k2       | 6,640 parámetros
 3: Conv       | 36,992 parámetros
 4: C3k2       | 26,080 parámetros
 5: Conv       | 147,712 parámetros
 6: C3k2       | 87,040 parámetros
 7: Conv       | 295,424 parámetros
 8: C3k2       | 346,112 parámetros
 9: SPPF       | 164,608 parámetros
10: C2PSA      | 249,728 parámetros
11: Upsample   | 0 parámetros
12: Concat     | 0 parámetros
13: C3k2       | 111,296 parámetros
14: Upsample   | 0 parámetros
15: Concat     | 0 parámetros
16: C3k2       | 32,096 parámetros
17: Conv       | 36,992 parámetros
18: Concat     | 0 parámetros
19: C3k2       | 86,720 parámetros
20: Conv       | 147,712 parámetros
21: Concat     | 0 parámetros
22: C3k2       | 378,880 parámetros
23: Detect     | 464,912 parámetros


# Entrenamiento

In [8]:

# ------------------------------------------------------------------
# 1. CONFIGURACIÓN BÁSICA
# ------------------------------------------------------------------
from pathlib import Path

ROOT = Path().resolve()
DATA_DIR = ROOT / "datasets" / "coco128"         # destino del dataset
ZIP_URL  = "https://github.com/ultralytics/assets/releases/download/v0.0.0/coco128.zip"
YAML_OUT = ROOT / "coco128_custom.yaml"          # YAML que crearemos
MODEL_WEIGHTS = "yolov8n.pt"                     # puedes cambiar a yolov8s.pt, etc.
EPOCHS = 20                                      # pon 50–100 para entreno serio
IMG_SIZE = 640
FREEZE_LAYERS = 10                               # cómo de profundo congelar (ver 4‑A)


In [9]:
# ------------------------------------------------------------------
# 2. DESCARGAR Y DESCOMPRIMIR COCO128
# ------------------------------------------------------------------
def download_and_extract(url: str, dest: Path):
    dest.parent.mkdir(parents=True, exist_ok=True)
    zip_path = dest.with_suffix(".zip")

    if dest.exists():
        print("Dataset ya existe – saltando descarga.")
        return

    # Descargar con barra de progreso
    print(f"Descargando {url.split('/')[-1]} …")
    r = requests.get(url, stream=True)
    total = int(r.headers.get("content-length", 0))
    with open(zip_path, "wb") as f, tqdm(
        desc="Descargando",
        total=total,
        unit="B",
        unit_scale=True,
        unit_divisor=1024,
    ) as bar:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)
            bar.update(len(chunk))

    # Descomprimir
    print("Descomprimiendo…")
    with zipfile.ZipFile(zip_path, "r") as z:
        z.extractall(dest.parent)  # crea carpeta coco128

    zip_path.unlink()  # eliminar zip
    print("Descarga y extracción completas.")


In [10]:

# ------------------------------------------------------------------
# 3. CREAR YAML DE CONFIGURACIÓN
# ------------------------------------------------------------------
def create_yaml(data_root: Path, yaml_path: Path):
    cfg = {
        "path": str(data_root),                   # raíz del dataset
        "train": "images/train2017",
        "val": "images/train2017",                # COCO128 usa las mismas 128 imágenes
        "test": "",                               # opcional
        # Lista de clases COCO
        "names": {
            0: "person", 1: "bicycle", 2: "car", 3: "motorcycle", 4: "airplane", 5: "bus",
            6: "train", 7: "truck", 8: "boat", 9: "traffic light", 10: "fire hydrant",
            11: "stop sign", 12: "parking meter", 13: "bench", 14: "bird", 15: "cat",
            16: "dog", 17: "horse", 18: "sheep", 19: "cow", 20: "elephant", 21: "bear",
            22: "zebra", 23: "giraffe", 24: "backpack", 25: "umbrella", 26: "handbag",
            27: "tie", 28: "suitcase", 29: "frisbee", 30: "skis", 31: "snowboard",
            32: "sports ball", 33: "kite", 34: "baseball bat", 35: "baseball glove",
            36: "skateboard", 37: "surfboard", 38: "tennis racket", 39: "bottle",
            40: "wine glass", 41: "cup", 42: "fork", 43: "knife", 44: "spoon", 45: "bowl",
            46: "banana", 47: "apple", 48: "sandwich", 49: "orange", 50: "broccoli",
            51: "carrot", 52: "hot dog", 53: "pizza", 54: "donut", 55: "cake", 56: "chair",
            57: "couch", 58: "potted plant", 59: "bed", 60: "dining table", 61: "toilet",
            62: "tv", 63: "laptop", 64: "mouse", 65: "remote", 66: "keyboard",
            67: "cell phone", 68: "microwave", 69: "oven", 70: "toaster", 71: "sink",
            72: "refrigerator", 73: "book", 74: "clock", 75: "vase", 76: "scissors",
            77: "teddy bear", 78: "hair drier", 79: "toothbrush",
        },
    }
    with open(yaml_path, "w") as f:
        yaml.safe_dump(cfg, f)
    print(f"YAML creado en {yaml_path}")



In [11]:
# ------------------------------------------------------------------
# 4. ENTRENAR EL MODELO (FINE‑TUNING)
# ------------------------------------------------------------------
def fine_tune(yaml_path: Path, freeze_depth: int):
    print("Cargando modelo…")
    model = YOLO(MODEL_WEIGHTS)

    # freeze = 10          # int  ➜ congela los 10 primeros módulos
    # freeze = [0,1,2,3]   # list ➜ congela los módulos cuyos índices estén en la lista
    # freeze = 0           # no congela nada


    print(f"Iniciando fine‑tuning {EPOCHS} épocas, freeze={freeze_depth}")
    model.train(
        data=str(yaml_path),
        epochs=EPOCHS,
        imgsz=IMG_SIZE,
        batch=-1,          # autodetecta batch óptimo
        freeze=freeze_depth,
        workers=4,         # ajusta según CPU
        lr0=0.001,         # LR inicial
        patience=5,        # early‑stopping
        verbose=True,
    )
    return model


In [12]:
# 5. VALIDAR E INFERIR
# ------------------------------------------------------------------
def validate_and_infer(model: YOLO, yaml_path: Path):
    print("Validación…")
    metrics = model.val(data=str(yaml_path))
    print(metrics)  # mAP50, mAP50‑95…

    # Elegimos imagen aleatoria del val‑set
    val_imgs = list((DATA_DIR / "images" / "train2017").glob("*.jpg"))
    img_path = random.choice(val_imgs)
    print(f"Inferencia en {img_path.name}")
    res = model(img_path)
    res[0].show()  # abre ventana (Mac/Linux: may need `res[0].save()`)

In [14]:
torch.manual_seed(0)  # reproducibilidad

download_and_extract(ZIP_URL, DATA_DIR)


Dataset ya existe – saltando descarga.


In [17]:
create_yaml(DATA_DIR, YAML_OUT)

📝 YAML creado en C:\Users\agodoy\Desktop\OneDrive_1_13-5-2025\coco128_custom.yaml


In [ ]:
yolo_model = fine_tune(YAML_OUT, FREEZE_LAYERS)

Cargando modelo…


100%|██████████| 6.25M/6.25M [00:00<00:00, 13.8MB/s]


Iniciando fine‑tuning 20 épocas, freeze=10
Ultralytics 8.3.133  Python-3.11.9 torch-2.7.0+cpu CPU (AMD Ryzen 5 5625U with Radeon Graphics)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\Users\agodoy\Desktop\OneDrive_1_13-5-2025\coco128_custom.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=10, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer

train: Scanning C:\Users\agodoy\Desktop\OneDrive_1_13-5-2025\datasets\coco128\labels\train2017... 126 images, 2 backgrounds, 0 corrupt: 100%|██████████| 128/128 [00:00<00:00, 144.22it/s]

train: New cache created: C:\Users\agodoy\Desktop\OneDrive_1_13-5-2025\datasets\coco128\labels\train2017.cache


...

train: Scanning C:\Users\agodoy\Desktop\OneDrive_1_13-5-2025\datasets\coco128\labels\train2017.cache... 126 images, 2 backgrounds, 0 corrupt: 100%|██████████| 128/128 [00:00<?, ?it/s]
C:\Users\agodoy\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


val: Fast image access  (ping: 0.10.0 ms, read: 145.767.2 MB/s, size: 53.4 KB)


val: Scanning C:\Users\agodoy\Desktop\OneDrive_1_13-5-2025\datasets\coco128\labels\train2017.cache... 126 images, 2 backgrounds, 0 corrupt: 100%|██████████| 128/128 [00:00<?, ?it/s]
C:\Users\agodoy\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Plotting labels to runs\detect\train\labels.jpg... 
WARNING 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.001' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000119, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\detect\train
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/8 [00:00<?, ?it/s]

In [ ]:
validate_and_infer(yolo_model, YAML_OUT)